## import 

In [1]:
%load_ext autoreload
%autoreload 2

In [58]:
import pkscreener

In [60]:
dir(pkscreener)

['Imports',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'find_spec']

In [2]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta
from tqdm import tqdm
import util_fun as uf
from enctoken import get_kite
kite = get_kite()
import sqlite3
from swifter import set_defaults
set_defaults(
    progress_bar=False,

)
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', str)

In [9]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

i=1
error_list = []
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
inst_filter = inst_filter[inst_filter.segment == 'NSE']
# inst_filter.query("Symbol == 'HDFCBANK'")

nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))
# inst_dict = dict(zip(inst_filter.Symbol, inst_filter.instrument_token))

In [10]:
len(inst_dict)

486

In [34]:
today_date = "2024-02-28"
start_dt = "2024-02-01"
end_dt = "2024-02-28"
time_frame = "5minute"

# import sql
# conn = sql.create_connection('../data/Nifty_500.db')
# cursor_obj = conn.cursor()
# sql.delete_table(conn,'fivteen_minute')
df_all = pd.DataFrame()
for symbol, instument in tqdm(inst_dict.items()):
    if symbol == "TCS":
        if i:
            try:
                df_all = df_all.append(
                    uf.get_data_parllel(
                        kite, instument, time_frame, start_dt, end_dt
                    ).assign(stock=symbol)
                )
        
                # df_day.to_sql('fivteen_minute',conn, if_exists="append",index = True, index_label = 'Date')
                # directory = f'../data/historical/{time_frame}/{symbol}'
                # if not os.path.exists(directory):
                #     os.makedirs(directory)
                # df_day.to_parquet(f'{directory}/part0.parquet')
            except Exception as e:
                print(instument,e)
                error_list.append(instument)
        
        i += 1
print(error_list)


100%|██████████| 486/486 [00:01<00:00, 378.01it/s]

[]


In [ ]:
# if i want to see breakout in 5 mins then 
# i need to fetch 5 mins data
# and check the consoldation and then breakout
# other thing i can do is to fetch 1 day data and then check the breakout in 5 mins

# but i think first step is to test the logic weather i can correctly identify the breakout or not


In [5]:
xx

NameError: name 'xx' is not defined

In [38]:
df_all.to_parquet(f'tcs_5min.parquet')
df_all = pd.read_parquet(f'tcs_5min.parquet')

In [39]:
# df_all.to_parquet('feb_5mins.parquet')
# df_all = pd.read_parquet('feb_5mins.parquet')
df_all = df_all[df_all.stock.isin(nifty_500.Symbol.unique())]
df_all = df_all[(df_all['Close'] > 80) &(df_all['Close'] < 5000) ]

In [40]:
df_all.reset_index(inplace=True)
df_all['Date']  = pd.to_datetime(df_all['Date']).dt.tz_localize(None)
df_all['date_only'] = pd.to_datetime(df_all['Date'].dt.date)

In [41]:
df_tcs= df_all.query("stock == 'TCS'")

# consolidate v2

In [19]:
4096- 3996

100

In [20]:
100/3996

0.025025025025025027

In [23]:
import pandas as pd
import mplfinance as mpf

# Load data
# Assuming data is in 'data.csv' with columns 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'
# Make sure to adjust the path to where you have your CSV file
df = df_tcs.copy()

# Define consolidation finding parameters
consolidation_percentage = 2.5 # Percentage defining the consolidation range (e.g., 2.5%)
min_periods_consolidated = 10   # Minimum number of periods for the consolidation

# Calculate the price range within each period as a percentage
df['RangePct'] = ((df['High'] - df['Low']) / df['Low']) * 100

# Find periods where the price range is within the consolidation percentage
consolidation = df['RangePct'] < consolidation_percentage


In [26]:
df[consolidation]

,Date,Open,High,Low,Close,Volume,stock,date_only,RangePct
511927,2024-02-01 09:15:00,3820.0,3828.0,3805.05,3819.65,118552,TCS,2024-02-01,0.6031458193716197
511928,2024-02-01 09:20:00,3819.65,3835.35,3819.15,3831.95,78982,TCS,2024-02-01,0.4241781548250217
511929,2024-02-01 09:25:00,3830.8,3833.75,3826.0,3826.55,36550,TCS,2024-02-01,0.2025614218504966
511930,2024-02-01 09:30:00,3826.55,3841.0,3823.6,3840.15,56882,TCS,2024-02-01,0.4550685218119074
511931,2024-02-01 09:35:00,3840.95,3852.15,3840.75,3851.0,99179,TCS,2024-02-01,0.2968170279242359
...,...,...,...,...,...,...,...,...,...
513272,2024-02-26 15:05:00,4000.0,4000.15,3998.6,4000.0,46794,TCS,2024-02-26,0.03876356724854154
513273,2024-02-26 15:10:00,4000.0,4000.2,3998.45,3999.95,54334,TCS,2024-02-26,0.043766959696882546
513274,2024-02-26 15:15:00,3999.95,4003.0,3997.05,4002.95,50519,TCS,2024-02-26,0.1488597840907624
513275,2024-02-26 15:20:00,4002.95,4005.4,4001.9,4003.15,42764,TCS,2024-02-26,0.08745845723281441


In [ ]:

# Find where the price has been consolidating for at least min_periods_consolidated
consolidating = consolidation.rolling(window=min_periods_consolidated).sum() == min_periods_consolidated

# Now find where the consolidation ends with a breakout
# A breakout occurs when the close price goes above the high or below the low of the consolidation range
breakout_up = (df['Close'] > df[consolidating]['High'].shift(1)) & consolidating
breakout_down = (df['Close'] < df[consolidating]['Low'].shift(1)) & consolidating

# Add the breakouts to the dataframe
df['BreakoutUp'] = breakout_up
df['BreakoutDown'] = breakout_down

# Filter the dataframe to only include breakout periods
breakouts = df[(df['BreakoutUp'] | df['BreakoutDown'])]

# Plotting
# We can use mplfinance to plot the entire dataframe and highlight the breakouts
apds = [mpf.make_addplot(df['BreakoutUp'], type='scatter', markersize=200, marker='^', color='g'),
        mpf.make_addplot(df['BreakoutDown'], type='scatter', markersize=200, marker='v', color='r')]

mpf.plot(df, type='candle', addplot=apds)


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 511927 to 513276
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1350 non-null   datetime64[ns]
 1   Open       1350 non-null   float64       
 2   High       1350 non-null   float64       
 3   Low        1350 non-null   float64       
 4   Close      1350 non-null   float64       
 5   Volume     1350 non-null   int64         
 6   stock      1350 non-null   object        
 7   date_only  1350 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(1)
memory usage: 127.2+ KB


,Date,Open,High,Low,Close,Volume,stock,date_only


In [55]:
import pandas as pd

import pandas as pd

def find_consolidations_and_breakouts(df, consolidation_candles=10, breakout_candle=1, consolidation_range_pct=2.0):
    consolidation_ranges = []
    breakouts = []

    # Iterate over the DataFrame
    for i in range(len(df) - (consolidation_candles + breakout_candle - 1)):
        # Define the window for the current consolidation check
        window = df.iloc[i:i+consolidation_candles]
        max_high = window['High'].max()
        min_low = window['Low'].min()

        # Calculate the consolidation range as a percentage
        consolidation_range = (max_high - min_low) / min_low * 100

        # Check if the consolidation range is within the specified percentage
        if consolidation_range <= consolidation_range_pct:
            # Check if the next candle after the consolidation window is a breakout
            next_candle = df.iloc[i+consolidation_candles]
            if next_candle['Close'] > max_high or next_candle['Close'] < min_low:
                # We have a breakout
                breakouts.append(i + consolidation_candles)
                consolidation_ranges.append((i, max_high, min_low))

    return consolidation_ranges, breakouts

# Assume `df` is your DataFrame with the OHLC data

df = df_tcs.copy()
df = df.query("date_only >= '2024-02-26'").reset_index()
# Assume `df` is your DataFrame with the OHLC data
consolidation_ranges, breakouts = find_consolidations_and_breakouts(df)

# Now, you can print the results or add markers to a plot to indicate where the consolidations and breakouts occurred
print("Consolidation Ranges (Start Index, Max High, Min Low):")
print(consolidation_ranges)
print("\nBreakouts (Index):")
print(breakouts)


Consolidation Ranges (Start Index, Max High, Min Low):
[(2, 4025.95, 4001.1), (15, 4006.45, 3993.25), (19, 4002.65, 3990.05), (30, 3999.95, 3989.0), (44, 3999.9, 3985.0), (65, 4008.0, 3995.75), (66, 4046.0, 3995.75), (71, 4072.0, 3997.05), (77, 4081.9, 4050.7), (89, 4083.0, 4070.5)]

Breakouts (Index):
[12, 25, 29, 40, 54, 75, 76, 81, 87, 99]


In [56]:
# filter index based on the breakouts
df.iloc[breakouts]

,index,Date,Open,High,Low,Close,Volume,stock,date_only
12,1287,2024-02-26 10:15:00,4006.25,4007.0,3993.95,3996.75,24075,TCS,2024-02-26
25,1300,2024-02-26 11:20:00,3994.8,3994.8,3990.05,3993.0,8152,TCS,2024-02-26
29,1304,2024-02-26 11:40:00,3991.0,3991.0,3987.05,3989.65,22139,TCS,2024-02-26
40,1315,2024-02-26 12:35:00,3990.0,3990.3,3982.75,3986.8,14768,TCS,2024-02-26
54,1329,2024-02-26 13:45:00,3996.5,4004.25,3996.5,4003.6,17259,TCS,2024-02-26
75,1350,2024-02-27 09:15:00,3999.0,4046.0,3999.0,4034.65,207304,TCS,2024-02-27
76,1351,2024-02-27 09:20:00,4034.65,4061.95,4033.0,4050.7,132885,TCS,2024-02-27
81,1356,2024-02-27 09:45:00,4065.25,4074.0,4065.0,4072.8,50615,TCS,2024-02-27
87,1362,2024-02-27 10:15:00,4079.7,4087.35,4078.75,4083.75,42625,TCS,2024-02-27
99,1374,2024-02-27 11:15:00,4072.5,4072.5,4064.85,4069.4,22696,TCS,2024-02-27


## consolidating 

In [ ]:
df_all.reset_index(inplace=True)
df_all['Date']  = pd.to_datetime(df_all['Date']).dt.tz_localize(None)
df_all['date_only'] = pd.to_datetime(df_all['Date'].dt.date)

## day level

In [ ]:
def is_consolidating(df):
    recent_closes = df[-7:]

    max_close = recent_closes["Close"].max()
    min_close = recent_closes["Close"].min()
    # print("the max close {} and min close {}".format(max_close, min_close))
    if min_close > (max_close * 0.985):
        # print("is consolidating {}".format(set(recent_closes['stock'])))
        return True
    return False


def is_breaking_out(df):
    last_close = df[-1:]["Close"].values[0]

    if is_consolidating(df[:-1]):
        recent_closes = df[-8:-1]
        if last_close > recent_closes["High"].max():
            print("is breaking {}".format(set(recent_closes["stock"])))
            breakout_up.extend(list(set(recent_closes["stock"])))
            return True
        if last_close < recent_closes["Low"].min():
            print("is breaking  below {}".format(set(recent_closes["stock"])))
            breakout_below.extend(list(set(recent_closes["stock"])))
            return True
    return False


In [ ]:
breakout_up = []
breakout_below = []
df_breakout = df_all.groupby("stock").apply(is_breaking_out).reset_index()

is breaking {'ABB'}
is breaking  below {'APTUS'}
is breaking {'CRISIL'}
is breaking {'MARICO'}
is breaking  below {'ONGC'}
is breaking {'RELIANCE'}
is breaking {'TIINDIA'}


In [ ]:
df_breakout[df_breakout[0] == True].stock.unique()

array(['ABB', 'APTUS', 'CRISIL', 'MARICO', 'ONGC', 'RELIANCE', 'TIINDIA'],
      dtype=object)

In [ ]:
breakout_below

[]

In [ ]:
breakout_below

['AEGISCHEM', 'APTUS', 'ONGC', 'SIS', 'STARHEALTH']

## minute lavel

In [51]:
def is_consolidating(df):
    recent_closes = df
    # print(recent_closes)

    max_close = recent_closes["Close"].max()
    min_close = recent_closes["Close"].min()
    # print("the max close {} and min close {}".format(max_close, min_close))
    if min_close > (max_close * 0.997):
        # print("is consolidating {}".format(set(recent_closes['stock'])))
        return True
    return False


def is_breaking_out(df):
    df_stock_date = pd.DataFrame()
    k = 0
    for i in range(10, 25):
        if is_consolidating(df[k:i]):
            last_close = df[i : i + 1]["Close"].values[0]
            recent_closes = df[k:i]
            # print(recent_closes)
            if last_close > recent_closes["High"].max():
                print("is breaking {}".format(set(recent_closes["stock"])))
                breakout_up.extend(list(set(recent_closes["stock"])))
                print(
                    "Date",
                    df[i : i + 1]["Date"].values[0],
                    "stock",
                    df[i : i + 1]["stock"].values[0],
                )

                # return True
            if last_close < recent_closes["Low"].min():
                print("is breaking  below {}".format(set(recent_closes["stock"])))
                breakout_below.extend(list(set(recent_closes["stock"])))
                print(
                    "Date",
                    df[i : i + 1]["Date"].values[0],
                    "stock",
                    df[i : i + 1]["stock"].values[0],
                )
                # df_stock_date = df_stock_date.append(
                #     pd.DataFrame(
                #         {
                #             "Date": recent_closes["Date"].unique()[0],
                #             "stock": recent_closes["stock"].unique()[0],
                #         }
                #     ),
                #     index=[0],
                # )
                # return True
            k += 1
        # return False


In [52]:
breakout_up = []
breakout_below = []
breakout_up_dict = {}
breakout_below_dict = {}
df_stock_date= pd.DataFrame()
df_breakout = df_all.groupby('stock').apply(is_breaking_out).reset_index()

In [54]:
df_breakout

,index


In [53]:
breakout_up

[]

In [ ]:
df_breakout[df_breakout[0] == True].stock.unique()

array(['ABBOTINDIA', 'ADANITRANS', 'ASIANPAINT', 'ATUL', 'BAJAJHLDNG',
       'BANDHANBNK', 'CENTURYTEX', 'CERA', 'CGPOWER', 'DELTACORP',
       'ENGINERSIN', 'GICRE', 'GRANULES', 'GRAPHITE', 'HAL', 'HCLTECH',
       'IDBI', 'IRCTC', 'JSWSTEEL', 'LEMONTREE', 'MCDOWELL-N', 'QUESS',
       'SHREECEM', 'SUMICHEM', 'TANLA', 'TATACHEM'], dtype=object)

In [ ]:
breakout_up

['AXISBANK',
 'BPCL',
 'ESCORTS',
 'ICICIBANK',
 'ICICIBANK',
 'INFY',
 'PETRONET',
 'PETRONET',
 'POLYCAB',
 'SHOPERSTOP',
 'TCS']

In [ ]:
breakout_below

['BLUEDART',
 'BLUEDART',
 'BPCL',
 'CLEAN',
 'HDFC',
 'HDFCBANK',
 'HINDUNILVR',
 'ICICIBANK',
 'LATENTVIEW',
 'PETRONET',
 'THYROCARE',
 'THYROCARE',
 'VOLTAS']